# Notebook 1:  Web Scraping

In this notebook we scrape www.basketball-reference.com for NBA player stats and salaries.
Our scraping workflow has two parts:
1.  **Past Years Data**: Using free agent lists for the last five years, scrape each listed player's previous season stats and his salary the following year.  These will form the features and targets, respectively, for our player market value model.
2.  **Current Year Data**: Using the entire list of players who have played in the 2021-22 season, scrape each player's current season stats and salary.  The stats will be used in our web app to predict each player's current market value based on their stats, which can be compared with his actual current season salary to calculate his surplus value.

## Imports 

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup, Comment
import requests
import lxml
import unicodedata
import json, pickle
import html5lib

## Utility Functions

In [3]:
def strip_accents_and_punctuation(text):
    '''Normalize player name spellings'''
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text).replace('.','').replace(',','').replace("'",'')


def update_row_with_dict(df,d,idx):
    '''Update the entries of a row in a dataframe, 
       using a dictionary to supply the values'''
    for key in d.keys():
        df.loc[idx, key] = d.get(key)

# (1) Scrape Player Stats and Salaries: Past Years


## Scrape individual player stats and salaries


In [19]:
fa_year=2023

In [20]:
def scrape_player(playerurl, fa_year):
    '''Get salaries and per-game stats from basketball-reference.com
       Edge case if a player played on multiple teams in a year: 
           — stats: his total stats are given across all teams ('Tm'='TOT') 
           — salary: his salaries are concatenated (eg: "$27,957,238$794,536")
    '''
    
    d = {}
    prev_season = str(fa_year-1) + '-' + str(fa_year-2000)
    next_season = str(fa_year) + '-' + str(fa_year-2000+1)
    
    playerresponse = requests.get(playerurl)
    playerpage = playerresponse.text
    playersoup = BeautifulSoup(playerpage, "lxml")

    ## Get Height and Weight:
    
    try:
        script_text = playersoup.find('script',{'type': 'application/ld+json'}).getText()
        biodata = json.loads(script_text) # a dictionary!
        weight = biodata['weight']['value'].replace('lbs','').strip()
        height = biodata['height']['value']
    except:
        weight = np.nan
        height = np.nan
        
    d['Weight']= weight
    d['Height']= height

    ## Get Basic Per-Game Stats:
    
    dfpergame = pd.read_html(str(playersoup.find(id='per_game')))[0]
    dfpergame = dfpergame.drop_duplicates(subset=['Season'])
    dfpergame = dfpergame.set_index('Season')

    featurelist = ['Age', 'Tm', 'G', 'GS', 'MP', 'FG%', '3P', '3P%', '3PA', 'FT', 'FT%','FTA', 'ORB', 'TRB', 
                    'AST', 'STL', 'BLK', 'TOV', 'PTS']
    
    for feature in featurelist:
        try: 
            d[feature] = dfpergame.loc[prev_season, feature] 
        except: 
            d[feature] = np.nan

    ## Get Advanced Per-Game Stats:  

    dfadvanced = pd.read_html(str(playersoup.find(id='advanced')))[0]
    dfadvanced = dfadvanced.drop_duplicates(subset=['Season'])
    dfadvanced = dfadvanced.set_index('Season')
    
    featurelist = ['USG%', 'TS%', 'PER', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']
    
    for feature in featurelist:
        try: 
            d[feature] = dfadvanced.loc[prev_season, feature] 
        except: 
            d[feature] = np.nan

    ## Get previous and next year's salary
    #  — If the player was on one team, it will be a string that looks like "$27,093,019"
    #  — If the player was on two teams, it will be a string that looks like "$27,957,238$794,536"
    #  — For 2021 free agents, Next Salary is incorrect (need 2021-22 from current stats)
    
    placeholder = playersoup.select_one('#all_all_salaries .placeholder')
    comment = next(elem for elem in placeholder.next_siblings if isinstance(elem, Comment))
    table = BeautifulSoup(comment, 'lxml')

    dfsalaries = pd.read_html(str(table.find(id="all_salaries")))[0]
    
    try: 
        d['PrevSal'] = dfsalaries[dfsalaries['Season'] == prev_season]['Salary'].sum()
    except: 
        d['PrevSal'] = np.nan
    try: 
        d['NextSal'] = dfsalaries[dfsalaries['Season'] == next_season]['Salary'].sum()
    except: 
        d['NextSal'] = np.nan
               
    return d

### Example player:

In [21]:
playerurl = 'https://www.basketball-reference.com/players/j/jamesle01.html'

fayear = 2023    # we get his stats for the previous season 202-2023
d = scrape_player(playerurl, fayear)
print(d)

{'Weight': '250', 'Height': '6-9', 'Age': 38.0, 'Tm': 'LAL', 'G': 55.0, 'GS': 54.0, 'MP': 35.5, 'FG%': 0.5, '3P': 2.2, '3P%': 0.321, '3PA': 6.9, 'FT': 4.6, 'FT%': 0.768, 'FTA': 5.9, 'ORB': 1.2, 'TRB': 8.3, 'AST': 6.8, 'STL': 0.9, 'BLK': 0.6, 'TOV': 3.2, 'PTS': 28.9, 'USG%': 33.3, 'TS%': 0.583, 'PER': 23.9, 'OWS': 3.2, 'DWS': 2.4, 'WS': 5.6, 'WS/48': 0.138, 'OBPM': 5.5, 'DBPM': 0.6, 'BPM': 6.1, 'VORP': 4.0, 'PrevSal': 0, 'NextSal': 0}


## Scrape stats and salaries for all free agents in a given year

In [32]:
def scrape_fa_year(fa_year):
    '''Find all free agents in a given year and call the function "scrape_player" 
       for each of them to get their previous season stats

      
       https://www.basketball-reference.com/friv/free_agents.cgi?year=
    '''

    FA_url = 'https://www.basketball-reference.com/contracts/players.html' + str(fa_year)
    prev_season = str(fa_year-1) + '-' + str(fa_year-2000)

    response = requests.get(FA_url)
    page = response.text
    FAsoup = BeautifulSoup(page, "lxml")

    table = FAsoup.find('table')
    rows = [row for row in table.find_all('tr')]  # tr tag is for rows
    rows_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
        
    df = pd.DataFrame()

    for i in range(1,len(rows)):
        try:
            print("Scraping row", i)
            # Rest of the code for scraping and processing the row

        except:
            print("Error occurred for row", i)
            continue
        try:
            name = rows_data[i][0]
            name_year = (name + ' ' + str(fa_year-1)).replace(' ','_')  #prev year
            name_year = strip_accents_and_punctuation(name_year)
             
            pos = rows_data[i][1]         #Pos
            fatype = rows_data[i][3]     #Type (UFA, RFA)
            oldteam = rows_data[i][4]    #OTm 
            prevstats = rows_data[i][5]  #Previous Year Stats  ('Did not play' if didn't play)
            newteam = rows_data[i][7]    #NTm

            nameid = rows[i].find_all('td')[0].find('a')['href']

            d = {'Name': name,  'Pos': pos, 'Type': fatype, 'OTm': oldteam, 
                 'PrevStats': prevstats, 'NTm' : newteam, 'ID': nameid}
            update_row_with_dict(df,d,name_year)
        
        
            playerurl = 'https://www.basketball-reference.com' + nameid
            playerdict = scrape_player(playerurl, fa_year)
            
            update_row_with_dict(df,playerdict,name_year)
            
        except:
            continue
            
    df['PrevYear'] = fa_year-1
    df.index.name = 'NameYear'
    
    return df

## Scrape last years of free agent stats and salaries

In [33]:
df2020 = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2020_merged.csv')
df2021 = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2021_merged.csv')
df2022 = pd.read_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_merged.csv')


In [34]:
# This takes 1-2 minutes per year

#df2016 = scrape_fa_year(2017)
#df2017 = scrape_fa_year(2018)
#df2018 = scrape_fa_year(2019)
#df2019 = scrape_fa_year(2020)
df2020 = scrape_fa_year(2021)

#added the following 2 lines of code
df2021 = scrape_fa_year(2022)
df2022 = scrape_fa_year(2023)

df2022.head(3)
# Note that for df2020 in particular, NextSal data cannot be obtained this way and is just listed as "0.0".  
# We will obtain in in a different way in Notebook 2 and populate the field in Notebook 3

AttributeError: 'NoneType' object has no attribute 'find_all'

# (2) Scrape Player Stats and Salaries:  Current Year

## Scrape player stats:  current year

In [35]:
##  Per-Game Stats

url_pergame = 'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'

def scrape_current_season_stats_pergame(url):
    '''Get current season stats for all players from basketball-reference.com
    '''
    d = {}
  
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    df = pd.read_html(str(soup.find(id='all_per_game_stats')))[0]
    return df

##  Advanced Stats

url_advanced = 'https://www.basketball-reference.com/leagues/NBA_2023_advanced.html'

def scrape_current_season_stats_advanced(url):
    '''Get current season stats for all players from basketball-reference.com
    '''
    d = {}
  
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    df = pd.read_html(str(soup.find(id='advanced_stats')))[0]
    return df

dfpergame = scrape_current_season_stats_pergame(url_pergame)
dfadvanced = scrape_current_season_stats_advanced(url_advanced)

## Merge per-game and advanced stats

cols_to_use = dfadvanced.columns.difference(dfpergame.columns)
dfcurrentstats = pd.merge(dfpergame, dfadvanced[cols_to_use], left_index=True, right_index=True, how='outer')
dfcurrentstats = dfcurrentstats.drop(['Unnamed: 19','Unnamed: 24'], axis=1)
print(dfcurrentstats.columns)
dfcurrentstats.sample(5)

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '3PAr',
       'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM', 'ORB%',
       'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP', 'WS',
       'WS/48'],
      dtype='object')


,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48
269,202,Sam Hauser,SF,25,BOS,80,8,16.1,2.3,5.0,...,1.8,12.7,1.1,6.9,8.6,.634,14.7,1.0,3.4,.128
53,39,Patrick Beverley,PG,34,CHI,22,22,27.5,2.1,5.4,...,0.3,10.1,1.9,12.5,10.0,.506,10.5,0.3,1.2,.096
423,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48
330,249,Nikola Jokić,C,27,DEN,69,69,33.7,9.4,14.8,...,11.2,31.5,1.8,17.0,20.2,.701,27.2,8.8,14.9,.308
395,299,Kevin Love,PF,34,CLE,41,3,20.0,2.7,7.0,...,0.5,13.6,0.4,13.1,20.1,.555,19.6,0.7,1.9,.112


## Scrape player salaries:  current year

In [36]:
def scrape_team(team):
    '''Returns dataframe with 2021-22 salary for all players from basketball-reference.com
    '''
    
    url = 'https://www.basketball-reference.com/contracts/' + team + '.html'
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")

    df = pd.read_html(str(soup.find(id='contracts')))[0]
        
    df.columns = df.columns.to_flat_index()
    df.rename(columns= {df.columns[0]: 'Name', df.columns[2]: 'CurrentSalary'}, inplace=True)
    df['CurrentTeam'] = team

    df = df.drop(df.tail(1).index)
    df = df[['Name','CurrentSalary','CurrentTeam']]
    
    df = df.dropna()
    
    return df

test = scrape_team('BOS')
display(test)

,Name,CurrentSalary,CurrentTeam
0,Kristaps Porziņģis,"$36,016,200",BOS
1,Jayson Tatum,"$32,600,060",BOS
2,Jaylen Brown,"$30,723,215",BOS
3,Malcolm Brogdon,"$22,500,000",BOS
4,Derrick White,"$18,107,143",BOS
5,Robert Williams,"$11,812,501",BOS
6,Al Horford,"$10,000,000",BOS
7,Payton Pritchard,"$4,037,277",BOS
8,Luke Kornet,"$2,413,304",BOS
9,Sam Hauser,"$1,927,896",BOS


In [37]:
tm_to_team =  {
 'TOR': 'Toronto Raptors',         'MEM': 'Memphis Grizzlies',
 'MIA': 'Miami Heat',              'BRK': 'Brooklyn Nets',
 'NOP': 'New Orleans Pelicans',    'MIL': 'Milwaukee Bucks',
 'CLE': 'Cleveland Cavaliers' ,    'LAL': 'Los Angeles Lakers',
 'ORL': 'Orlando Magic',           'HOU': 'Houston Rockets' ,
 'WAS': 'Washington Wizards' ,     'PHO': 'Phoenix Suns',
 'UTA': 'Utah Jazz',               'SAC': 'Sacramento Kings',
 'CHO': 'Charlotte Hornets',       'CHI': 'Chicago Bulls' ,
 'NYK': 'New York Knicks',         'DEN': 'Denver Nuggets' ,
 'PHI': 'Philadephia 76ers' ,      'SAS': 'San Antonio Spurs' ,
 'LAC': 'Los Angeles Clippers',    'OKC': 'Oklahoma City Thunder' ,
 'MIN': 'Minnesota Timberwolves',  'DET': 'Detroit Pistons' ,
 'IND': 'Indiana Pacers',          'GSW': 'Golden State Warriors' ,
 'POR': 'Portland Trailblazers',   'ATL': 'Atlanta Hawks',
 'BOS': 'Boston Celtics',          'DAL':'Dallas Mavericks',
 }

teams = list(tm_to_team.keys())  #We just need the team names as used in the urls

df = pd.DataFrame(columns=['Name', 'CurrentSalary', 'CurrentTeam'])

for team in teams:
    df_team = scrape_team(team)
    df = pd.concat([df, df_team])
    
df.sample(5)

,Name,CurrentSalary,CurrentTeam
7,Julian Strawther,"$2,431,080",DEN
17,Dewayne Dedmon,"$2,866,667",DET
17,Joshua Primo,"$4,341,600",SAS
3,Landry Shamet,"$10,250,000",WAS
6,Otto Porter Jr.,"$6,300,000",TOR


There are a 16 duplicate names (same player, multiple teams/salaries).  We deal with it manually, keeping the entry corresponding to the actual current team.

# df[df.duplicated(subset = ['Name'])].sort_values("Name")
ids = df["Name"]
df[ids.isin(ids[ids.duplicated()])].sort_values("Name")

mask1 = ((df.Name == 'Armoni Brooks') & ~(df.CurrentTeam == 'TOR'))
mask2 = ((df.Name == 'Blake Griffin') & ~(df.CurrentTeam == 'BRK'))
mask3 = ((df.Name == 'D.J. Augustin') & ~(df.CurrentTeam == 'LAL'))
mask4 = ((df.Name == 'Danuel House Jr.') & ~(df.CurrentTeam == 'UTA'))
mask5 = ((df.Name == 'DeAndre Jordan') & ~(df.CurrentTeam == 'PHI'))
mask6 = ((df.Name == "DeAndre' Bembry") & ~(df.CurrentTeam == 'MIL'))
mask7 = ((df.Name == 'DeMarcus Cousins') & ~(df.CurrentTeam == 'DEN'))
mask8 = ((df.Name == 'Dewayne Dedmon') & ~(df.CurrentTeam == 'MIA'))
mask9 = ((df.Name == 'Goran Dragić') & ~(df.CurrentTeam == 'BRK'))
mask10 = ((df.Name == 'Isaiah Hartenstein') & ~(df.CurrentTeam == 'LAC'))
mask11 = ((df.Name == 'Jevon Carter') & ~(df.CurrentTeam == 'MIL'))
mask12 = ((df.Name == 'Kemba Walker') & ~(df.CurrentTeam == 'NYK'))
mask13 = ((df.Name == 'Moses Brown') & ~(df.CurrentTeam == 'CLE'))
mask14 = ((df.Name == 'Nicolas Batum') & ~(df.CurrentTeam == 'LAC'))
mask15 = ((df.Name == 'Tomáš Satoranský') & ~(df.CurrentTeam == 'WAS'))
mask16 = ((df.Name == 'Tristan Thompson') & ~(df.CurrentTeam == 'CHI'))
mask17 = ((df.Name == 'Dewayne Dedmon') & ~(df.CurrentTeam == 'DET'))

df = df[~mask1 & ~mask2 & ~mask3 & ~mask4 & ~mask5 & ~mask6 & ~mask7  & ~mask8 & ~mask9
           & ~mask10 & ~mask11 & ~mask12 & ~mask13 & ~mask14  & ~mask15 & ~mask16 & ~mask17] 

dfcurrentsalaries = df.copy()
dfcurrentsalaries.shape

## Create player-to-url dictionary

* Get the player page url for any player who played in a game this past season (this is mostly useful for easily creating hyperlinks in the final web app, and could also potentially disambiguate identically named players)


In [39]:
url = 'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'

player_to_url = {}

response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, "lxml")


table = soup.find('table')
rows = [row for row in table.find_all('tr')]  # tr tag is for rows

for row in rows[1:]:
    try:
        name = row.findAll('td')[0].getText()
        nameid = row.findAll('td')[0].contents[0]['href']
        if name not in player_to_url:
            player_to_url[name] = nameid
    except:
        pass

{k: player_to_url[k] for k in list(player_to_url)[:5]}

{'Precious Achiuwa': '/players/a/achiupr01.html',
 'Steven Adams': '/players/a/adamsst01.html',
 'Bam Adebayo': '/players/a/adebaba01.html',
 'Ochai Agbaji': '/players/a/agbajoc01.html',
 'Santi Aldama': '/players/a/aldamsa01.html'}

In [40]:
dfplayer_to_url = pd.Series(player_to_url).to_frame().reset_index()
dfplayer_to_url.rename(columns= {'index': 'Name',0: 'ID'}, inplace=True)
dfplayer_to_url.head(5)

,Name,ID
0,Precious Achiuwa,/players/a/achiupr01.html
1,Steven Adams,/players/a/adamsst01.html
2,Bam Adebayo,/players/a/adebaba01.html
3,Ochai Agbaji,/players/a/agbajoc01.html
4,Santi Aldama,/players/a/aldamsa01.html


# Save All Scraped Data

df2016.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/df2016_raw.csv')
df2017.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/df2017_raw.csv')
df2018.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/df2018_raw.csv')
df2019.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/df2019_raw.csv')
df2020.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/df2020_raw.csv')

#added the following 2 lines of code
df2021.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/df2020_raw.csv')
df2022.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/df2020_raw.csv')

dfcurrentstats.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/dfcurrentstats.csv')
dfcurrentsalaries.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/dfcurrentsalaries.csv')

dfplayer_to_url.to_csv(r'/Users/richardsihombing/Documents/BigDataNBA/data/dfplayer_to_url.csv')

In [42]:
df.sample(10)

,Name,CurrentSalary,CurrentTeam
0,Stephen Curry,"$51,915,615",GSW
2,James Harden,"$35,640,000",PHI
4,Derrick White,"$18,107,143",BOS
3,Anthony Edwards,"$13,534,817",MIN
4,Malik Monk,"$9,945,830",SAC
6,Monte Morris,"$9,800,926",DET
2,Jarred Vanderbilt,"$4,698,000",LAL
4,Talen Horton-Tucker,"$11,020,000",UTA
5,Robert Covington,"$11,692,308",LAC
2,Marvin Bagley III,"$12,500,000",DET


In [44]:
df2020.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2020_raw.csv')

#added the following 3 lines of code
df2021.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2021_raw.csv')
df2022.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\df2022_raw.csv')


dfcurrentstats.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\dfcurrentstats.csv')
df.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\dfcurrentsalaries.csv')

dfplayer_to_url.to_csv(r'C:\Users\Vincenzo\Documents\Master 2. Semester\GutHub_Sofi\BigDataNBA\data\dfplayer_to_url.csv')